<a href="https://colab.research.google.com/github/Yoohwa-Kim/2021_K-DATA_project/blob/main/diary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브와 연동합니다
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요한 모듈을 설치합니다
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch #원래 ==1.3.1
!pip install emoji

#SKT에서 공개한 KoBERT 모델을 불러옵니다 
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 356.7 MB 31 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595732 sha256=ace144e4cbfc65086dc402fdf835a5163c7abf97ee7d6c05d929400fc9133e35
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.0 MB 4.3 MB/s 
     |████████████████████████████████| 311 kB 4.2 MB/s 
     |████████████████████████████████| 131 kB 47.9 MB/s 
     |████████████████████████████████| 895 kB 37.7 MB/s 
     |████████████████████████████████| 7.8 MB 29.3 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 138 kB 49.4 MB/s 
     |████████████

In [33]:
import pandas as pd

happy = pd.read_excel('/content/drive/My Drive/data/행복.xlsx')
fun = pd.read_excel('/content/drive/My Drive/data/즐거움.xlsx')
sad = pd.read_excel('/content/drive/My Drive/data/슬픔.xlsx')
anger = pd.read_excel('/content/drive/My Drive/data/분노.xlsx')


happy

,Unnamed: 0,keyWorld,raw_text
0,0,해피,히히 다정둥이 바바님🥰 바바님 텐션이 좋으셔서 항상 기분 좋아요! 해피바이러스 바...
1,1,해피,나 내 생일날 해피가랜드 입고싶은데 그러려면 사야함(칰쇼)
2,2,해피,사랑을 하고 있어... 지윤이 먹어주면 지영이 대박 해피함
3,3,해피,미첫내...? 방학숙제 많이 없는 사람 존나 해피함
4,4,해피,찾기 어려운것도 있지만 한번 사면 재구매같은걸로 꾸준히 사기도 편하고 빨리 오고 ...
...,...,...,...
5562,5563,기분 좋다,오늘은 좀 오래 게임하다가 누웠어 컬님이랑 오랜만에 이야기도 나눴어 컬님이랑 얘기하...
5563,5564,기분 좋다,근데 여기서 또 웃긴 건 티아 본인은 자신이 귀엽다는 자각이 없다는 거예요 원래 본...
5564,5565,기분 좋다,오히려 집에서 일하니까 기분이 좀 나아진다
5565,5566,기분 좋다,"...분위기가 좋긴 한데, 조금 묘하게 좋아서 이상한 기분입니다...(끊임없이 휘..."


In [36]:
#전처리를 위한 함수
import emoji

def extract_emojis(s):
  return ''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])

def preprocessing(data, label):
  import re
  dt = data['raw_text'].copy() #문장만 선택
  dt = dt.dropna() #결측치 제거
  dt = dt.drop_duplicates() #중복 제거
  sentences = dt.tolist()
  new_sent=[]
  for i in range(len(sentences)):
    sent = sentences[i] 
    emo=extract_emojis(sent)
    # print(sent)
    for k in emo:
      sent = re.sub(k,' ',sent)
    if type(sent) != str: # 문장 중 str 아닌 것 처리
      sent = str(sent)
    if len(sent) < 2: continue #길이 1 이상인 것만 선택
    sent = re.sub('\n',' ',sent)
    sent = re.sub('ㅋㅋ+','ㅋㅋ',sent)
    sent = re.sub('ㅠㅠ+','ㅠㅠ',sent)
    sent = re.sub('ㅇㅇ+','ㅇㅇ',sent) 
    sent = re.sub('ㄷㄷ+','ㄷㄷ',sent) 
    sent = re.sub('ㅎㅎ+','ㅎㅎ',sent)
    sent = re.sub('ㅂㅂ+','ㅂㅂ',sent) 
    sent = re.sub(';;;+',';;',sent)
    sent = re.sub('!!!+','!!',sent)
    sent = re.sub('~+','~',sent)
    sent = re.sub('[?][?][?]+','??',sent) 
    sent = re.sub('[.][.][.]+','...',sent) 
    sent = re.sub('[-=+,#/:$@*\"※&%ㆍ』\\‘|\(\)\[\]\<\>`\'…》]','',sent)
    new_sent.append(sent)
  dt = pd.DataFrame(pd.Series(new_sent), columns=['raw_text'])
  dt['emotion'] = label
  return dt

In [38]:
happy = preprocessing(happy, '행복')
fun = preprocessing(fun, '즐거움')
sad = preprocessing(sad, '슬픔')
anger = preprocessing(anger, '분노')

for i in [happy, fun, sad, anger]:
  print('1차 레이블 결과', i['emotion'][0],len(i))



1차 레이블 결과 행복 5400
1차 레이블 결과 즐거움 5274
1차 레이블 결과 슬픔 5655
1차 레이블 결과 분노 4356


In [39]:


# 각 감정별 키워드 데이터가 약 10개 씩으로 이루어져 있기 때문에 마지막 키워드에 대한 데이터 1000개를 평가 데이터로 선택
sentence_train = pd.concat([happy, fun, sad, anger], axis=0, ignore_index=True)


for i in ['행복','즐거움','슬픔','분노']:
  print('sentence_train',i,len(sentence_train[sentence_train['emotion'] == i]))


sentence_train 행복 5400
sentence_train 즐거움 5274
sentence_train 슬픔 5655
sentence_train 분노 4356


In [11]:
label

NameError: ignored

In [41]:
#모델에 입력하기 위해 형식을 맞춰줍니다
def label(x):
  if x=='행복': return 0.0
  elif x=='즐거움': return 1.0
  elif x=='슬픔': return 2.0
  elif x=='분노': return 3.0
  else: return x

sentence_train["emotion"] = sentence_train["emotion"].apply(label)


dtls = [list(sentence_train.iloc[i,:]) for i in range(len(sentence_train))]
dtls[:10] #형식이 통일되었습니다

[[' 히히 다정둥이 바바님  바바님 텐션이 좋으셔서 항상 기분 좋아요! 해피바이러스 바바님   바바님과 함께라면 언제나 꽃길! 바바님 행복한 하루 보내시기 ',
  0.0],
 ['나 내 생일날 해피가랜드 입고싶은데 그러려면 사야함칰쇼', 0.0],
 [' 사랑을 하고 있어... 지윤이 먹어주면 지영이 대박 해피함', 0.0],
 [' 미첫내...? 방학숙제 많이 없는 사람 존나 해피함', 0.0],
 [' 찾기 어려운것도 있지만 한번 사면 재구매같은걸로 꾸준히 사기도 편하고 빨리 오고 국내에서 구하는것들보다 성분 빵빵하고 싸고 세일 자주함  정도? 쇼핑에 공 좀 들이면 해피쇼핑 가능! 나는 샤워용품도 사서 애용함',
  0.0],
 ['텍스트 볼 때마다 폰트 이뻐서 해피함 기여어 텍스트!!', 0.0],
 [' ... 갑자기 멈춤 ㅋ 문양이나 옵젝이나 커미 좋은 거 없나 뒤적뒤젂... 흑흡 굿즈... 좋아요... 뽑을 생각하면 벌써 좋아서 굴러다님 헤헤 저두 눈에 띄는 거 잇음 물 ',
  0.0],
 ['어무니께 지금 헨젤이 매우 거지임을 어필하자 트릿 대신 통 사준대서 매우 해피함', 0.0],
 ['여기 하나 더 얹어서 썸머님 해피버스데이 .. 얼마전에 다시 정좽했는데 이마 너무 쳐서 뇌로 핀볼게임함', 0.0],
 ['해피신시사이저 master 풀콤 성공 행복함', 0.0]]

## 2 모델 투입

In [42]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

from kobert.utils import get_tokenizer 
from kobert.pytorch_kobert import get_pytorch_kobert_model 

from transformers import AdamW 
from transformers.optimization import WarmupLinearSchedule

In [43]:
##GPU 사용 시
device = torch.device("cuda:0")

In [44]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [45]:
#koBERT의 토크나이저를 사용합니다
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [46]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [47]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

## 3 학습

In [48]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1 
log_interval = 200 
learning_rate =  5e-5

In [52]:
# train, validation, test set을 나눠주세요
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(dtls, test_size=0.2, random_state=123)

In [53]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [54]:
#모델을 만들고 GPU 사용 설정을 해줍니다
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [55]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [56]:
#옵티마이저와 손실함수 설정
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

#정확도를 계산하기 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [57]:
#학습 과정
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad() 
        token_ids = token_ids.long().to(device) 
        segment_ids = segment_ids.long().to(device) 
        valid_length= valid_length 
        label = label.long().to(device) 
        out = model(token_ids, valid_length, segment_ids) 
        loss = loss_fn(out, label) 
        loss.backward() 
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) 
        optimizer.step() 
        scheduler.step()  # Update learning rate schedule 
        train_acc += calc_accuracy(out, label)  
        if batch_id % log_interval == 0: 
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1))) 
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1))) 
    model.eval() #모델 평가 부분 
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc(정확도) {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/259 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.4504296779632568 train acc 0.265625
epoch 1 batch id 201 loss 0.06064635515213013 train acc 0.7998289800995025
epoch 1 train acc 0.8374289468039469


  0%|          | 0/65 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:

# 차후 사용을 위해 학습된 모델을 저장했습니다
torch.save(model.state_dict(), 'drive/My Drive//kobert_ending_finale.pt')


## 4 평가

In [29]:
sentence_eval['emotion']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
3995    3.0
3996    3.0
3997    3.0
3998    3.0
3999    3.0
Name: emotion, Length: 4000, dtype: float64

In [ ]:
label

tensor([0, 1, 1, 3, 0, 3, 2, 2, 2, 1, 0, 1, 2, 1, 3, 1, 2, 0, 0, 1, 3, 2, 2, 1,
        2, 1, 3, 1, 1, 2, 0, 2, 1, 2], device='cuda:0')

In [30]:
##################################################
# 평가용 Test_set을 모델에 입력하기 위해 형식을 맞춰줍니다
# sentence_eval을 앞부분에 정의하니까 괜찮아짐..

dtls_eval = [list(sentence_eval.iloc[i,:]) for i in range(len(sentence_eval))]

data_test = BERTDataset(dtls_eval, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [31]:
# 해당 데이터에 대해 분류를 시작합니다
model.eval()
answer=[]
train_acc = 0.0
test_acc = 0.0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  max_vals, max_indices = torch.max(out, 1)
  answer.append(max_indices.cpu().clone().numpy())
  test_acc += calc_accuracy(out, label)
print('정답률: ',test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/63 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


정답률:  0.9010416666666666


In [32]:
# 제출 형식에 맞춰 파일을 저장해줍니다
ls = []
for i in answer:
  ls.extend(i)

pred = pd.DataFrame(ls, columns=['Predicted'])
df = pd.concat([sentence_eval['raw_text'], pred['Predicted'], sentence_eval['emotion']], axis=1) 

def test(x):
  if x==0.0: return '행복'
  elif x==1.0: return '즐거움'
  elif x==2.0: return '슬픔'
  elif x==3.0: return '분노'
  else: return x

df["Predicted"] = df["Predicted"].apply(test)
df["emotion"] = df["emotion"].apply(test)

for i in ['행복','즐거움','슬픔','분노']: 
  print(i, '개수', len(df[df['emotion'] == i]))
  print('예측 개수', len(df[df['emotion'] == i][df['Predicted'] == i]))
  print('정답률',len(df[df['emotion'] == i][df['Predicted'] == i])/len(df[df['emotion'] == i]))

행복 개수 1000
예측 개수 690
정답률 0.69
즐거움 개수 1000
예측 개수 963
정답률 0.963
슬픔 개수 1000
예측 개수 966
정답률 0.966
분노 개수 1000
예측 개수 982
정답률 0.982


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
